In [11]:
import os
from dotenv import load_dotenv
load_dotenv()

HUGGINGFACE_TOKEN = os.getenv("HUGGINGFACE_TOKEN")
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [10]:
# Document loader
from langchain_community.document_loaders import PyMuPDFLoader, DirectoryLoader

#PyMuPDF fastest in parsling(analyize the text)  #extract images from text in pdf -> ## pip install rapidocr-onnxruntime
loader = DirectoryLoader('Data/', glob="*.pdf", loader_cls=PyMuPDFLoader)  
data = loader.load()

# Extract text content from the 'Document' objects
texts = [doc.page_content for doc in data]
texts[40]

'Marvel’s The Avengers\x08\nPhotocopiable\nActivity worksheets\nLEVEL 2\nTeacher Support Programme\nenglish.com/englishreaders Published by Pearson Education Limited\nMarvel’s The Avengers – Activity worksheets\u2003 \u2003 2 of 2\n© 2018 MARVEL \nAfter reading\n10\t Write short answers to these questions about the \nTesseract.\na\t Where did Stark find the Tesseract?\n……………………………………………………\nb\t What did the Other want do with it?\n……………………………………………………\nc\t What did S.H.I.E.L.D. want to do with it?\n……………………………………………………\nd\t What did Loki want to do with it?\n……………………………………………………\ne\t Where is it at the end of the story?\n……………………………………………………\n11\t Write happy or unhappy.\nAfter the attack, how did these people in Manhattan \nfeel? \na\t “The Avengers saved us!” ……………………\nb\t “We can sleep safely in our beds because the \nAvengers are protecting us.” ……………………\nc\t “What aren’t the Avengers telling us about the \nattack?” ……………………\nd\t “Who’s going to pay? It was their fight—and \nwhere 

In [ ]:
# Text Splitter using Semantic chunking - which uses self-attenation concept in Transformer to create independent chunks
from langchain_experimental.text_splitter import SemanticChunker
from langchain_google_genai import GoogleGenerativeAIEmbeddings

embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

# Break Points
text_splitter = SemanticChunker(
    embeddings, breakpoint_threshold_type="percentile"
)

docs = text_splitter.create_documents(texts)

In [18]:
print(docs[41].page_content)

Golu is the festive display of dolls and figurines in South India during the autumn festive 
season. These displays are typically thematic, narrating a legend from a Hindu text to court 
life, weddings, everyday scenes and miniature kitchen utensils. Each displayed item in a Golu 
display is called a Golu doll. These are typically made by rural artisans from clay and wooden 
materials that are brightly painted.
